<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# Les générateurs 

* Montrer que si on appelle un générateur on reçoit un objet générateur
  * que c'est propice dans un for
  * mais que ça ne marche pas lorsqu'on veut cascader les appels

## Complément - niveau intermédiaire

In [ ]:
def dividers(n, verbose=False):
    for i in range(1, n):
        if n % i == 0:
            if verbose: 
                print(f'trouvé diviseur {i} de {n}')
            yield i

In [ ]:
dividers(28)

In [ ]:
dividers(28, verbose=True)

In [ ]:
for d in dividers(28):
    print(d)

In [ ]:
# 28 est un nombre parfait
sum(dividers(28))

## Complément - niveau avancé

Maintenant si un générateur appelle un générateur .. il ne se passe rien

In [ ]:
def dividers_square(n):
    for i in dividers(n):
        dividers(i)

In [ ]:
try:
    for i in dividers_square(28):
        print(i)
except Exception as e:
    print(f"OOPS {type(e)} {e}")

Et si on utilise juste `yield` ça ne fait pas du tout ce qu'on veut :

In [ ]:
def dividers_square(n):
    for i in dividers(n):
        yield dividers(i)

In [ ]:
try:
    for i in dividers_square(28):
        print(i)
except Exception as e:
    print(f"OOPS {type(e)} {e}")

Ce qu'il faut faire plutôt, c'est ceci :

In [ ]:
def dividers_square(n):
    for i in dividers(n):
        yield from dividers(i, verbose=True)

In [ ]:
try:
    for i in dividers_square(28):
        print(i)
except Exception as e:
    print(f"OOPS {type(e)} {e}")